In [ ]:
import os
import tensorflow as tf
import cProfile

In [ ]:
tf.executing_eagerly()

In [ ]:
x = [[2.]]
m = tf.matmul(x, x)
print('hello, {}'.format(m))

In [ ]:
a = tf.constant([[1, 2],
                 [3, 4]])
print(a)

In [ ]:
a

In [ ]:
a.numpy()

In [ ]:
b = tf.add(a, 1)
b

In [ ]:
print(a * b)

In [ ]:
print(tf.matmul(a, b))

In [ ]:
import numpy as np

In [ ]:
print(np.multiply(a, b))

In [ ]:
def fizzbuzz(max_num):
    counter = tf.constant(0)
    max_num = tf.convert_to_tensor(max_num)
    for num in range(1, max_num.numpy() + 1):
        if int(num % 3) == 0 and int(num % 5) == 0:
            print('FizzBuzz')
        elif int(num % 3) == 0:
            print('Fizz')
        elif int(num % 5) == 0:
            print('Buzz')
        else:
            print(num)
        counter += 1

In [ ]:
fizzbuzz(15)

In [ ]:
w = tf.Variable([[1.0]])

with tf.GradientTape() as tape:
    loss = w * w

grad = tape.gradient(loss, w)
print(grad)

In [ ]:
(mnist_images, mnist_labels), _ = tf.keras.datasets.mnist.load_data()

dataset = tf.data.Dataset.from_tensor_slices((tf.cast(mnist_images[..., tf.newaxis] / 255, tf.float32), tf.cast(mnist_labels, tf.int64)))
dataset = dataset.shuffle(1000).batch(32)

In [ ]:
mnist_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, [3, 3], activation='relu', input_shape=(None, None, 1)),
    tf.keras.layers.Conv2D(16, [3, 3], activation='relu'),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(10)
])

In [ ]:
for images, labels in dataset.take(1):
    logit = mnist_model(images)[0].numpy()
    print("Logit: ", logit)
    print("Prediction: {}, Label: {}".format(tf.argmax(logit), labels[0]))

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(True)

loss_history = []

In [ ]:
# 每一个训练步骤，传入一个batch的样本和标签
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # 使用当前模型计算结果
        logits = mnist_model(images, training=True)
        # 检查结果的形状（可不要）
        tf.debugging.assert_equal(logits.shape, (32, 10))
        # 计算loss
        loss_value = loss_object(labels, logits)
    
    # 将本次的loss记录下来
    loss_history.append(loss_value.numpy().mean())
    # 自动计算每个可训练参数的的梯度
    grads = tape.gradient(loss_value, mnist_model.trainable_variables)
    # 采用适当的优化器来调整参数
    optimizer.apply_gradients(zip(grads, mnist_model.trainable_variables))

In [ ]:
def train(epochs):
    for epoch in range(epochs):
        for (batch, (images, labels)) in enumerate(dataset):
            train_step(images, labels)
        
        print("Epoch {} finished.".format(epoch+1))

In [ ]:
train(epochs=3)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(28, 4))
plt.plot(loss_history)
plt.xlabel("Batch #")
plt.ylabel("Loss[entropy]")

In [ ]:
class Linear(tf.keras.Model):
    def __init__(self):
        super(Linear, self).__init__()
        
        self.W = tf.Variable(5., name='weight')
        self.B = tf.Variable(10., name='bias')
    
    def call(self, inputs):
        return self.W * inputs + self.B

In [ ]:
NUM_EXAMPLES = 2000
training_inputs = tf.random.normal([NUM_EXAMPLES])
noise = tf.random.normal([NUM_EXAMPLES])
training_outputs = training_inputs * 3 + 2 + noise

def loss(model, input, target):
    error = model(input) - target
    return tf.reduce_mean(tf.square(error))

def grad(model, input, target):
    with tf.GradientTape() as tape:
        loss_value = loss(model, input, target)
    
    return tape.gradient(loss_value, [model.W, model.B])

In [ ]:
model = Linear()
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

print("Initial Loss = {}".format(loss(model, training_inputs, training_outputs)))

steps = 1000
for i in range(steps):
    grads = grad(model, training_inputs, training_outputs)
    optimizer.apply_gradients(zip(grads, [model.W, model.B]))

    if i % 100 == 0:
        print("Loss at step {}: {}".format(i, loss(model, training_inputs, training_outputs)))

In [ ]:
print('W: {}, B: {}'.format(model.W.numpy(), model.B.numpy()))

In [ ]:
model.save_weights('weights')
status = model.load_weights('weights')

In [ ]:
x = tf.Variable(10.)
checkpoint = tf.train.Checkpoint(x=x)

In [ ]:
x.assign(2.)
x

In [ ]:
checkpoint_path = './ckpt/'
checkpoint.save(checkpoint_path)

In [ ]:
x.assign(11.)
x

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_path))
x

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, [3,3], activation='relu'),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(10)
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
checkpoint_path = 'path/to/model_dir'
if not os.path.exists(checkpoint_path):
    os.makedirs(checkpoint_path)

checkpoint_prefix = os.path.join(checkpoint_path, 'ckpt')
root = tf.train.Checkpoint(optimizer=optimizer, model=model)

root.save(checkpoint_prefix)
root.restore(tf.train.latest_checkpoint(checkpoint_prefix))

In [ ]:
m = tf.keras.metrics.Mean('loss')
m(0)
print(m.result())
m(5)
print(m.result())
m([8, 9])
print(m.result())

In [ ]:
logdir = './tb/'
writer = tf.summary.create_file_writer(logdir)

In [ ]:
steps = 1000
with writer.as_default():
    for i in range(steps):
        step = i + 1
        loss = 1 - 0.001 * step
        if step % 100 == 0:
            tf.summary.scalar('loss', loss, step=step)

In [ ]:
!ls tb

In [ ]:
def line_search_step(fn, init_x, rate=1.0):
    with tf.GradientTape() as tape:
        tape.watch(init_x)
        value = fn(init_x)
    
    grad = tape(value, init_x)
    grad_norm = tf.reduce_sum(grad * grad)
    init_value = value

    while value > init_value - rate * grad_norm:
        x = init_x - rate * grad
        value = fn(x)
        rate /= 2.0
    
    return x, value

In [ ]:
@tf.custom_gradient
def clip_gradient_by_norm(x, norm):
    y = tf.identity(x)
    
    def grad_fn(dresult):
        return [tf.clip_by_norm(dresult, norm), None]
    
    return y, grad_fn

In [ ]:
def log1pexp(x):
    return tf.math.log(1 + tf.exp(x))

def grad_log1pexp(x):
    with tf.GradientTape() as tape:
        tape.watch(x)
        value = log1pexp(x)
    
    return tape.gradient(value, x)

In [ ]:
grad_log1pexp(tf.constant([0., 0.])).numpy()

In [ ]:
grad_log1pexp(tf.constant([100., 0.])).numpy()

In [ ]:
@tf.custom_gradient
def log1pexp(x):
    e = tf.exp(x)
    def grad(dy):
        return dy * (1 - 1 / (1 + e))
    
    return tf.math.log(1 + e), grad

def grad_log1pexp(x):
    with tf.GradientTape() as tape:
        tape.watch(x)
        value = log1pexp(x)
    
    return tape.gradient(value, x)

In [ ]:
print(grad_log1pexp(tf.constant(100.)))

In [ ]:
import time

def measure(x, steps):
    tf.matmul(x, x)
    start = time.time()
    for i in range(steps):
        x = tf.matmul(x, x)
    _ = x.numpy()
    end = time.time()
    return end - start

shape = (1000, 1000)
steps = 200

print("Time to multiply a {} matrix by itself {} time".format(shape, steps))

with tf.device('/cpu:0'):
    print("CPU: {} secs.".format(measure(tf.random.normal(shape), steps)))

with tf.device('/gpu:0'):
    print("GPU: {} secs.".format(measure(tf.random.normal(shape), steps)))

In [ ]:
if tf.config.list_physical_devices("GPU"):
    x = tf.random.normal([10, 10])

    x_gpu0 = x.gpu()
    x_cpu = x.cpu()

    _ = tf.matmul(x_gpu0, x_gpu0)
    _ = tf.matmul(x_cpu, x_cpu)